# Packages

In [ ]:
# Matplotlib setup
from matplotlib_setup import mpl_setup

# Modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as font_manager
import matplotlib.gridspec as gridspec
import seaborn as sns

# Mute warnings
import warnings
warnings.filterwarnings("ignore")

# Setup

In [ ]:
# directories
wd = '/Users/lneumann/Documents/'
database_dir = wd + 'Products/ALMOND/dense_gas_letter/database/'
fits_dir = wd + 'Products/ALMOND/dense_gas_letter/fits/'
tab_dir =  wd + 'Products/ALMOND/dense_gas_letter/tables/'
plot_dir = wd + 'Products/ALMOND/dense_gas_letter/plots/main/'

# sample tables
PHANGS_sample_table_path = wd + 'Data/PHANGS/tables/phangs_sample_table_v1p6.csv'
ALMOND_sample_table_path = tab_dir + 'almond_and_empire_sample_table.csv'

# load sample table
sample_table = pd.read_csv(PHANGS_sample_table_path, comment='#', skiprows=1)
sample_table_almond = pd.read_csv(ALMOND_sample_table_path)

# alpha CO prescription
aco = 'SL24'
# aco = 'MW'

# SNR threshold for masking
SNR_threshold = 3
scatter_sigma = 3 # used to compute the scatter (set higher to eliminate contribution from noise)

# markers
marker_centre = '^'
marker_disc = 'o'
marker_agn = 'x'

# colors
scatter_alpha = 0.3
scatter_color = 'grey'
# color_centre = lighten_color('tab:purple', amount=0.7)
# color_disc = lighten_color('tab:green', amount=0.7)
color_centre = 'tab:orange'
color_disc = 'tab:cyan'
ec_darken = 2

# constants to compute surface densities and dense gas fraction
alpha_co10 = 4.35  # [M_Sun/pc^2/(K km/s)] (Bolatto et al. 2013)
alpha_HCN = 15  # [M_Sun/pc^2/(K km/s)] (Schinnerer & Leroy 2024)
c_dense = alpha_HCN/alpha_co10

# labels
xlabel_list = [r'$\Sigma_{\star}$ [M$_{\odot}\,\mathrm{pc}^{-2}$]',
               r'$\Sigma_{\rm mol}$  [M$_{\odot}\,\mathrm{pc}^{-2}$]',
               r'$P_{\rm DE}/k_{\rm B}$ [$\mathrm{K}\,\mathrm{cm}^{-3}$]']
ylabel_list = [r'$\frac{W_{\mathrm{HCN}(1-0)}\;[\mathrm{K}\,\mathrm{km}\,\mathrm{s}^{-1}]}{W_{\mathrm{CO}(1-0)}\;[\mathrm{K}\,\mathrm{km}\,\mathrm{s}^{-1}]}$',
               r'$\frac{\Sigma_\mathrm{SFR}\;[\mathrm{M}_{\odot}\,\mathrm{yr}^{-1}\,\mathrm{pc}^{-2}]}{W_{\mathrm{HCN}(1-0)}\;[\mathrm{K}\,\mathrm{km}\,\mathrm{s}^{-1}]}$']

# data files
xlabel_list_file = ['Sd_star', 'Sd_mol_'+aco, 'P_DE_'+aco]
ylabel_list_file = ['hcn_co21', 'sfr_hcn']
xdata_label_list = ['SD_star [Msun/pc2]', 'SD_mol (%s) [Msun/pc2]' % aco, 'P_DE (%s) [kB.K/cm3]' % aco]

# fitting file
xlabel_fitting_list = ['Sd_star', 'Sd_mol_'+aco, 'P_DE_'+aco]
ylabel_fitting_list = ['hcn_co', 'sfr_hcn']

# number of galaxies in panel
# ngalaxies_list = ['31 galaxies\n' + r'@ $\sim 1-2\,$kpc', '31 galaxies\n' + r'@ $\sim 1-2\,$kpc', '26 galaxies\n' + r'@ $\sim 1-2\,$kpc']
ngalaxies_list = ['31 galaxies', '31 galaxies', '26 galaxies']


# secondary x-axis for CO from Sigma_mol
def Sdmol_to_Wco(Sd_mol):
    return Sd_mol / alpha_co10
def Wco_to_Sdmol(W_co21):
    return W_co21 * alpha_co10
    
# secondary y-axis for fdense from HCN/CO
def Wratio_to_fdense(Wratio):
    return Wratio * c_dense
def fdense_to_Wratio(fdense):
    return fdense / c_dense
    
# secondary y-axis for SFE from SFR/HCN
def sfr_hcn_ratio_to_sfedense(sfr_hcn_ratio):
    return sfr_hcn_ratio / alpha_HCN  # units: Myr^-1
def sfedense_to_sfr_hcn_ratio(sfedense):
    return sfedense * alpha_HCN

# secondary y-axis for fdense from HCN/CO
def Wratio_log_to_fdense(Wratio):
    return 10**Wratio * c_dense
def fdense_to_Wratio_log(fdense):
    return 10**np.log10(fdense / c_dense) 

# secondary y-axis for SFE from SFR/HCN
def sfr_hcn_ratio_log_to_sfedense(sfr_hcn_ratio):
    return 10**sfr_hcn_ratio / alpha_HCN  # units: Myr^-1
def sfedense_to_sfr_hcn_ratio_log(sfedense):
    return 10**np.log10(sfedense * alpha_HCN)

# functions for violin plots
def adjacent_values(vals, q1, q3):
    upper_adjacent_value = q3 + q3 - q1 * 1.5
    upper_adjacent_value = np.clip(upper_adjacent_value, q3, vals[-1])

    lower_adjacent_value = q1 - (q3 - q1) * 1.5
    lower_adjacent_value = np.clip(lower_adjacent_value, vals[0], q1)
    return lower_adjacent_value, upper_adjacent_value

# box for labels
box = dict(boxstyle='round', facecolor='white', edgecolor='k', alpha=0.8)  # set box

# HCN/CO, SFR/HCN vs Molecular Cloud Properties (centre vs disc)

In [ ]:
savepng = False
savepdf = True

# show density contours
density_contours = True

######################################

# plotting settings
xlim_list = [(0.9*3e1, 6e3), (0.9*5e0, 4e2), (0.9*1e4, 5e6)]
ylim_list = [(5e-3, 1.5e-1), (2e-8, 8e-7)]

# get number of rows and columns
nrows = len(ylabel_list)
ncols = len(xlabel_list)

######################################

# matplotlib setup
mpl_setup(figtype='paper-1/1', ms=4)

# create figure
fig = plt.figure()

# gridspec inside gridspec
gs = gridspec.GridSpec(1, 2, figure=fig, wspace=0, width_ratios=[6,1])
gs1 = gs[0].subgridspec(nrows, ncols, wspace=0, hspace=0) # left grid (relations)
gs2 = gs[1].subgridspec(2, 1, wspace=0, hspace=0) # right grid (distributions)

######################################
# loop over subplots
for i in range(nrows*ncols):
    
    # make subplot
    ax = fig.add_subplot(gs1[i])
    # ax = fig.add_subplot(nrows, ncols, i+1)
    
    # get column and row indeces
    id_row = i // ncols
    id_col = i % ncols

    # labels
    xlabel_file = xlabel_list_file[id_col]
    ylabel_file = ylabel_list_file[id_row]
    xdata_label = xdata_label_list[id_col]
    xlabel_fitting = xlabel_fitting_list[id_col]
    ylabel_fitting = ylabel_fitting_list[id_row]
        
    # data range
    xlim = xlim_list[id_col]
    ylim = ylim_list[id_row]

    # make data arrays
    x, x_err =  np.array([]), np.array([])
    y, y_err = np.array([]), np.array([])
    y_ul, y_ll = np.array([]), np.array([])
        
    ########## DISC STACKS ############

    # data file
    fname_table = database_dir + 'almond_and_empire_database_stacks_%s_wo_centres.csv' % xlabel_file
    df = pd.read_csv(fname_table)
    df = df[df['Survey']=='ALMOND']
    
    # get data
    xdata = df[xdata_label]
    xdata_err = df['e_' + xdata_label]
    hcn = df['W_HCN [K.km/s]']
    hcn_err = df['e_W_HCN [K.km/s]']
    co = df['W_CO10 [K.km/s]']
    co_err = df['e_W_CO10 [K.km/s]']
    sfr = df['SD_SFR [Msun/yr/kpc2] (no inclination correction)'] * 1e-6  # Msun/yr/pc2
    sfr_err = df['e_SD_SFR [Msun/yr/kpc2] (no inclination correction)'] * 1e-6 # Msun/yr/pc2
    if ylabel_file == 'hcn_co21':
        ydata = hcn/co
        ydata_err = np.abs(ydata) * np.sqrt((hcn_err/hcn)**2 + (co_err/co)**2)
    elif ylabel_file == 'sfr_hcn':
        ydata = sfr/hcn
        ydata_err = np.abs(ydata) * np.sqrt((hcn_err/hcn)**2 + (sfr_err/sfr)**2)

    # compute limits
    snr = ydata/ydata_err
    mask = (snr >= SNR_threshold)
    
    ydata_ul = np.full_like(ydata, np.nan)
    ydata_ll = np.full_like(ydata, np.nan)
    
    if ylabel_file == 'hcn_co21':
        ydata_ul[~mask] = hcn_err[~mask] * SNR_threshold / co[~mask]
            
    elif ylabel_file == 'sfr_hcn':
        ydata_ll[~mask] = sfr[~mask] / (hcn_err[~mask] * SNR_threshold)

    # append to array
    x = np.concatenate((x, xdata))
    x_err = np.concatenate((x_err, xdata_err))
    y = np.concatenate((y, ydata))
    y_err = np.concatenate((y_err, ydata_err))
    y_ul = np.concatenate((y_ul, ydata_ul))
    y_ll = np.concatenate((y_ll, ydata_ll))
        
    ########## PLOTTING ############

    # plot binned data
    handle_disc = ax.errorbar(xdata[mask], ydata[mask], yerr=ydata_err[mask], marker=marker_disc, mfc=color_disc, mec='k', mew=0.5, 
                              ecolor=color_disc, ls='none', elinewidth=0.5, capsize=0, label='Disc', zorder=5)

    # plot upper limits
    if ylabel_file == 'hcn_co21':
        (_, caps, _) = ax.errorbar(xdata[~mask], ydata_ul[~mask], yerr=ydata_ul[~mask]/10, uplims=True, linestyle='none',
                                   marker='None', elinewidth=0.5, color=color_disc, zorder=4)
    # plot lower limits
    elif ylabel_file == 'sfr_hcn':
        (_, caps, _) = ax.errorbar(xdata[~mask], ydata_ll[~mask], yerr=ydata_ll[~mask]/6, lolims=True, linestyle='none',
                                      marker='None', elinewidth=0.5, color=color_disc, zorder=4)
    for cap in caps:
        cap.set_color(color_disc)
        cap.set_ms(3)
        cap.set_mew(0)

    # density contours
    if density_contours:
        sns.kdeplot(x=xdata[mask], y=ydata[mask], log_scale=True, levels=[0.25, 0.5, 0.75, 1], color=color_disc, fill=True, alpha=0.5, zorder=1, ax=ax)
        sns.kdeplot(x=xdata[mask], y=ydata[mask], log_scale=True, levels=[0.25, 0.5, 0.75, 1], color=color_disc, zorder=2, ax=ax)
        
    ########## MEDIANS ############
    
    # if id_col == (ncols-1):
    if id_col == 0:
        
        # create subplot
        ax1 = fig.add_subplot(gs2[id_row])

        # violin plot
        parts = ax1.violinplot(np.log10(ydata[mask]), positions=[1], showmeans=False, showmedians=False, showextrema=False, widths=1)
        
        for pc in parts['bodies']:
            pc.set_facecolor(color_disc)
            pc.set_edgecolor('black')
            pc.set_alpha(1)

        # percentile
        quartile1, median, quartile3 = np.percentile(np.log10(ydata[mask]), [25, 50, 75])
        whiskers = np.array(adjacent_values(np.sort(np.log10(ydata[mask])), quartile1, quartile3))
        whiskers_min, whiskers_max = whiskers[0], whiskers[1]
        ax1.vlines(1, quartile1, quartile3, color='k', linestyle='-', lw=5)
        ax1.vlines(1, whiskers_min, whiskers_max, color='k', linestyle='-', lw=1)
        ax1.scatter(1, median, marker=marker_disc, c='white', s=10, zorder=3)
        
        # median (including censored data)    
        median_cens = np.nanmedian(np.log10(ydata))
        ax1.scatter(1, median_cens, marker='_', c=color_disc, s=20, zorder=3)

    
    ########## CENTRES ############
    
    # data file
    fname_table = database_dir + 'almond_and_empire_database_sightlines_centres.csv'
    df = pd.read_csv(fname_table)
    
    # exclude doublets
    df = df[(df['Galaxy']!='ngc0628') | (df['Survey']!='EMPIRE')]
    df = df[(df['Galaxy']!='ngc2903') | (df['Survey']!='EMPIRE')]
    df = df[(df['Galaxy']!='ngc4321') | (df['Survey']!='EMPIRE')]
    
    # get data
    xdata = df[xdata_label]
    xdata_err = df['e_' + xdata_label]
    hcn = df['W_HCN [K.km/s]']
    hcn_err = df['e_W_HCN [K.km/s]']
    co = df['W_CO10 [K.km/s]']
    co_err = df['e_W_CO10 [K.km/s]']
    sfr = df['SD_SFR [Msun/yr/kpc2] (no inclination correction)'] * 1e-6  # Msun/yr/pc2
    sfr_err = df['e_SD_SFR [Msun/yr/kpc2] (no inclination correction)'] * 1e-6 # Msun/yr/pc2
    if ylabel_file == 'hcn_co21':
        ydata = hcn/co
        ydata_err = np.abs(ydata) * np.sqrt((hcn_err/hcn)**2 + (co_err/co)**2)
    elif ylabel_file == 'sfr_hcn':
        ydata = sfr/hcn
        ydata_err = np.abs(ydata) * np.sqrt((hcn_err/hcn)**2 + (sfr_err/sfr)**2)
    
    # compute limits
    snr = ydata/ydata_err
    mask = (snr >= SNR_threshold)
    
    ydata_ul = np.full_like(ydata, np.nan)
    ydata_ll = np.full_like(ydata, np.nan)
    
    if ylabel_file == 'hcn_co21':
        ydata_ul[~mask] = hcn_err[~mask] * SNR_threshold / co[~mask]
            
    elif ylabel_file == 'sfr_hcn':
        ydata_ll[~mask] = sfr[~mask] / (hcn_err[~mask] * SNR_threshold)
    
    # plotting
    handle_centre = ax.errorbar(xdata[mask], ydata[mask], yerr=ydata_err[mask], marker=marker_centre, mfc=color_centre, mec='k', mew=0.5, 
                                ecolor=color_centre, ls='none', elinewidth=0.5, capsize=0, label='Centre', zorder=10)
    
    # bar
    # bar = np.array([sample_table_almond['Bar'][np.where(sample_table_almond['Galaxy']==glxy)[0][0]] for glxy in df['Galaxy']])
    # bar_mask = (bar=='Y') & mask
    # handle_bar = ax.scatter(xdata[bar_mask], ydata[bar_mask], marker='s', fc='none', ec='k', lw=0.5, zorder=11, label='Bar')
    
    # agn 
    agn = np.array([sample_table_almond['AGN'][np.where(sample_table_almond['Galaxy']==glxy)[0][0]] for glxy in df['Galaxy']]) 
    agn_mask = (agn=='Y') & mask
    handle_agn = ax.scatter(xdata[agn_mask], ydata[agn_mask], marker=marker_agn, c='k', lw=0.5, zorder=11, label='AGN')
    
    # density contours
    if density_contours:
        sns.kdeplot(x=xdata[mask], y=ydata[mask], log_scale=True, levels=[0.25, 0.5, 0.75, 1], color=color_centre, fill=True, alpha=0.5, zorder=1, ax=ax)
        sns.kdeplot(x=xdata[mask], y=ydata[mask], log_scale=True, levels=[0.25, 0.5, 0.75, 1], color=color_centre, zorder=2, ax=ax)
    
    ########## MEDIANS ############

    # if id_col == (ncols-1):
    if id_col == 0:

        # centres violin plot
        parts = ax1.violinplot(np.log10(ydata[mask]), positions=[2], showmeans=False, showmedians=False, showextrema=False, widths=1)
        
        for pc in parts['bodies']:
            pc.set_facecolor(color_centre)
            pc.set_edgecolor('black')
            pc.set_alpha(1)

        # Centres percentile
        quartile1, median, quartile3 = np.percentile(np.log10(ydata[mask]), [25, 50, 75])
        whiskers = np.array(adjacent_values(np.sort(np.log10(ydata[mask])), quartile1, quartile3))
        whiskers_min, whiskers_max = whiskers[0], whiskers[1]
        ax1.vlines(2, quartile1, quartile3, color='k', linestyle='-', lw=5)
        ax1.vlines(2, whiskers_min, whiskers_max, color='k', linestyle='-', lw=1)
        ax1.scatter(2, median, marker=marker_centre, c='white', s=10, zorder=3)
        print('Centre', median)
        # BAR percentile
        # bar_mask = (bar=='Y') & mask
        # quartile1, median, quartile3 = np.percentile(np.log10(ydata[bar_mask]), [25, 50, 75])
        # whiskers = np.array(adjacent_values(np.sort(np.log10(ydata[bar_mask])), quartile1, quartile3))
        # whiskers_min, whiskers_max = whiskers[0], whiskers[1]
        # ax1.vlines(3, quartile1, quartile3, color='k', linestyle='-', lw=5)
        # ax1.vlines(3, whiskers_min, whiskers_max, color='k', linestyle='-', lw=1)
        # ax1.scatter(3, median, marker='s', c='white', s=10, zorder=3)       

        # AGN percentile
        agn_mask = (agn=='Y') & mask
        quartile1, median, quartile3 = np.percentile(np.log10(ydata[agn_mask]), [25, 50, 75])
        whiskers = np.array(adjacent_values(np.sort(np.log10(ydata[agn_mask])), quartile1, quartile3))
        whiskers_min, whiskers_max = whiskers[0], whiskers[1]
        ax1.vlines(3, quartile1, quartile3, color='k', linestyle='-', lw=5)
        ax1.vlines(3, whiskers_min, whiskers_max, color='k', linestyle='-', lw=1)
        ax1.scatter(3, median, marker=marker_agn, c='white', s=10, zorder=3)   
        print('AGN', median)

        # non-AGN
        quartile1, median, quartile3 = np.percentile(np.log10(ydata[~agn_mask]), [25, 50, 75])
        print('non-AGN', median)

    
    ########## FITTING ##############

    # load fit results
    fname_fitting = fits_dir + 'almond_and_empire_fits_' + ylabel_fitting + '_vs_' + xlabel_fitting + '.csv'
    df = pd.read_csv(fname_fitting)
    intercept = df['intercept'][0]
    slope = df['slope'][0]
    x_off = df['x-axis offset'][0]
    y_scatter = df['scatter (residuals)'][0]

    # plot linmix median line fit
    x_fit = np.logspace(np.log10(xlim[0]), np.log10(xlim[1]), 10)
    y_fit = 10**(intercept + slope * (np.log10(x_fit)-x_off))
    ax.plot(x_fit, y_fit, ls='solid', c='k', lw=2, zorder=1)
    
    # plot scatter
    y_sca_up = 10**(intercept + slope * (np.log10(x_fit)-x_off) - y_scatter)
    y_sca_dw = 10**(intercept + slope * (np.log10(x_fit)-x_off) + y_scatter)
    ax.fill_between(x_fit, y_sca_up, y_sca_dw, color=scatter_color, lw=0, alpha=scatter_alpha)
    ax.plot(x_fit, y_sca_up, ls='dashed', color='k', lw=1, zorder=1)
    ax.plot(x_fit, y_sca_dw, ls='dashed', color='k', lw=1, zorder=1)

    
    ############### axis settings #######################
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    ax.set_xscale('log')
    ax.set_yscale('log')
    if id_col > 0:
        ax.tick_params(labelleft=False, labelright=False)

    if id_col == (ncols-1):
        
        # secondary y-axis
        def log_to_lin(x):
            return 10**x
        def lin_to_log(x):
            return np.log10(x)
            
        if id_row == 0:
            # create new log-scale y-axis (HCN/CO)
            secyax11 = ax1.secondary_yaxis('left', functions=(log_to_lin, lin_to_log)) # secondary y-axis
            secyax11.set_yscale('log')
            secyax11.tick_params(labelleft=False, labelbottom=False)
            # create secondary axis for f_dense
            secyax12 = ax1.secondary_yaxis('right', functions=(Wratio_log_to_fdense, fdense_to_Wratio_log)) # secondary y-axis for f_dense
            secyax12.set_yscale('log')
            secyax12.set_ylabel(r'$f_\mathrm{dense}$')
        
        elif id_row == 1:
            # create new log-scale y-axis (SFR/HCN)
            secyax11 = ax1.secondary_yaxis('left', functions=(log_to_lin, lin_to_log)) # secondary y-axis
            secyax11.set_yscale('log')
            secyax11.tick_params(labelleft=False, labelbottom=False)
            # create secondary axis for SFE_dense
            secyax12 = ax1.secondary_yaxis('right', functions=(sfr_hcn_ratio_log_to_sfedense, sfedense_to_sfr_hcn_ratio_log)) # secondary y-axis for f_dense
            secyax12.set_yscale('log')
            secyax12.set_ylabel(r'$\mathrm{SFE}_\mathrm{dense}\;[\mathrm{yr}^{-1}]$')
        
        ax1.set_ylim(np.log10(ylim))
        # ax1.tick_params(which='both', bottom=False, top=False, left=False, right=False, labelbottom=False, labelleft=False)
        ax1.tick_params(which='both', top=False, left=False, right=False, labelleft=False)
        ax1.set_xlim(0,4)
        ax1.set_xticks(ticks=[1,2,3], labels=['Disc', 'Centre', 'AGN'])
        ax1.tick_params(axis='x', rotation=90)

    ############ axis labels ############

    # x-axis labels
    if id_row == (nrows-1):
        ax.set_xlabel(xlabel_list[id_col])
    else:
        ax.tick_params(labelbottom=False)
        ax.set_xlabel('')

    # y-axis labels
    if id_col == 0:
        ax.set_ylabel(ylabel_list[id_row], size='x-large')
    else:
        ax.tick_params(labelleft=False)  
        ax.set_ylabel('')

    ############ annotations ############
    if id_col == (ncols-1):
        mpl.rcParams['text.usetex'] = False
        font = font_manager.FontProperties(family='Arial', style='normal', size=6)
        arrowprops = dict(arrowstyle="-|>", color='k', shrinkA=0.5, shrinkB=4, lw=1)
        if id_row == 0:
            ax1.annotate('median\n' + r'($\mathrm{S/N}\geq 3$)', xy=(1, -1.72), xytext=(3, -2), arrowprops=arrowprops, ha='center', fontproperties=font)
            ax1.annotate('median\n(all S/N)', xy=(1, -1.83), xytext=(2.4, -2.2), arrowprops=arrowprops, ha='center', fontproperties=font)
        # if id_row == 1:
        #     ax1.text(3, -6.35, 'Medians and\n25th to 75th\npercentiles', ha='center', fontproperties=font, bbox=box)
        mpl.rcParams['text.usetex'] = True

    if id_row == 0:
        mpl.rcParams['text.usetex'] = False
        font = font_manager.FontProperties(family='Arial', style='normal', size=6)
        ax.text(0.75, 0.1, ngalaxies_list[id_col], ha='center', va='center', transform=ax.transAxes, fontproperties=font)
        mpl.rcParams['text.usetex'] = True
        
    if (id_col == (ncols-1)) & (id_row == 0):
        mpl.rcParams['text.usetex'] = False
        font = font_manager.FontProperties(family='Arial', style='normal', size=6)
        ax1.text(0.5, 0.95, '31 galaxies', ha='center', va='center', transform=ax1.transAxes, fontproperties=font)
        mpl.rcParams['text.usetex'] = True

    
    ############ legend ############
    
    if (id_row == 0) & (id_col == 0):
        mpl.rcParams['text.usetex'] = False
        font = font_manager.FontProperties(family='Arial', style='normal', size=8)
        legend1 = ax.legend(loc='upper left', prop=font, handles=[handle_disc, handle_centre, handle_agn])
        legend1.set_zorder(50)
        mpl.rcParams['text.usetex'] = True

################################

# save plot
savepath = plot_dir + 'HCN_scaling_relations_centre_vs_disc'
if savepng:
    plt.savefig(savepath + '.png')
if savepdf:
    plt.savefig(savepath + '.pdf')

plt.show()

In [ ]:
10**-1.281203 / 10**-1.4550

In [ ]:
10**-7.05779 / 10**-7.2281